<a href="https://colab.research.google.com/github/Marieta3/sentiment-analysis-russian/blob/master/sentiment_analiza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analiza sentimenta tvitova na ruskom jeziku


##Importovanje potrebnih biblioteka

In [0]:
import tensorflow as tf
import pandas as pd
from nltk.stem.snowball import RussianStemmer
from nltk.tokenize import TweetTokenizer
import re
from collections import Counter
from datetime import datetime

##Konstante

In [0]:
VOCAB_SIZE = 5000

##Ucitavanje podataka

In [0]:
col = 3 #kolona koja sadrzi tvit, ostale kolone nisu potrebne

positive = pd.read_csv(
    'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/positive.csv', header=None, delimiter=';')[[col]];
negative = pd.read_csv(
    'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/negative.csv', header=None, delimiter=';')[[col]];


##Testiranje ucitanih podataka


In [4]:
print('Pozitivnih tvitova: ' +str(len(positive)))
print('Negativnih tvitova: ' +str(len(negative)))

Pozitivnih tvitova: 114911
Negativnih tvitova: 111923


##Izvlacenje osnovnog oblika iz tokena

In [5]:
stemmer = RussianStemmer()
#stem_cache={}
def get_stem(token):
  #stem =stem_cache.get(token, None)
  #if stem:
  #  return stem
  
  token=re.sub('[^а-яА-Я ]', '', token)
  stem=stemmer.stem(token)
  #stem_cache[token]=stem
  return stem
get_stem('!')

''

##Tokenizacija

In [0]:
recnik = Counter()
tokenizer = TweetTokenizer()


def popuni_recnik(tweets):
  for t in tweets.values.tolist():
    #print(t[0])
    tweet=t[0]
    
    tokens=tokenizer.tokenize(tweet)
    for token in tokens:
      stem=get_stem(token)
      recnik[stem]+=1
    
start_time=datetime.now()  
popuni_recnik(positive)
popuni_recnik(negative)
end_time=datetime.now()

In [14]:
print('Potrebno vreme za popunjavanje recnika: ', end_time-start_time)
print("Ukupan broj razlicitih reci: ", len(recnik))
vocab = sorted(recnik, key=recnik.get, reverse=True)[:VOCAB_SIZE]  #sortirano po broju ponavljanja opadajuce, prvih 5000
print(vocab[:10])
print(vocab[1000:1020])
print(vocab[-10:])
print("Duzina skracenog recnika: ",len(vocab))

Potrebno vreme za popunjavanje recnika:  0:04:26.037069
Ukupan broj razlicitih reci:  91780
['', 'не', 'я', 'и', 'в', 'на', 'а', 'что', 'так', 'с']
['добав', 'хз', 'причин', 'пропа', 'биолог', 'придт', 'выбор', 'ух', 'глуп', 'помо', 'шерлок', 'ссылк', 'горяч', 'устро', 'прода', 'назва', 'солнц', 'молодежк', 'тест', 'руга']
['фитнес', 'ае', 'зураг', 'юма', 'обучен', 'пьем', 'гал', 'гденибуд', 'нац', 'частн']
Duzina skracenog recnika:  5000
